In [31]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from torch import embedding
from nltk.corpus import stopwords
from nltk.corpus import wordnet 
from nltk.corpus import stopwords
import nltk
import numpy as np
from sklearn import preprocessing

In [32]:
#subject6 的最明顯波長是68.4813s
#space = 17.12 #sample step size
#timeframe = 17.12
global q
q=0

In [33]:
from tracemalloc import start


def calculate(gcs,destination,timeframe,space,start_time=0,end=480): #所以gcs要是string

    
    df = pd.read_csv(gcs) 

    df['Start_time'] = pd.to_numeric(df['Start_time'])
    #df['end_time'] = pd.to_numeric(df['end_time'])
    global all_sentences001
    all_sentences001 = []
    t=start_time
    
    while t<end:
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+timeframe))].iterrows():
            sentences.append(rows["Words"])
        all_sentences001.append(" ".join(sentences))
        t+=space

    
    model = SentenceTransformer('all-MiniLM-L6-v2')
    global SBERT 
    SBERT= []

    def get_cosine_similarity(feature_vec_1, feature_vec_2):    
        return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]
    # Two lists of sentences
    for i in range(0,len(all_sentences001)-1):
        #Compute embedding for both lists
        embeddings1 = model.encode(all_sentences001[i], convert_to_tensor=True)
        #print(all_sentences001[i])
        embeddings2 = model.encode(all_sentences001[i+1], convert_to_tensor=True)
        #print(all_sentences001[i+1])
        #Compute cosine-similarities
        cosine_scores = util.cos_sim(embeddings1, embeddings2)
        #print(cosine_scores)
        # convert from PyTorch tensor to numpy array
        SBERT.append(get_cosine_similarity(embeddings1, embeddings2))
    ######
        print("timeframe: ",i,":",i+10,"\n")
        print(get_cosine_similarity(embeddings1, embeddings2),"\n")
        print(all_sentences001[i],"\n")
        print(all_sentences001[i+1],"\n")
        print()

    
    all_sentences001 = []
    stops = set(stopwords.words('english'))
    t=start_time
    while t <end:
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+timeframe))].iterrows():
            if rows["Words"].lower() not in stops:
                sentences.append(rows["Words"].lower())
                #print(rows["Words"])
        all_sentences001.append(" ".join(sentences))
        #print(t)
        t += space 
    ''' 舊的sample step
    for t in range(0,480,0.5):
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+10))].iterrows():
            if rows["Words"].lower() not in stops:
                sentences.append(rows["Words"].lower())
        all_sentences001.append(" ".join(sentences))
    '''    
    #nltk.download('omw-1.4')
    #nltk.download('wordnet')
    #nltk.download('stopwords')
    global pair_counts
    pair_counts = []
    stops = set(stopwords.words('english'))
    global all_sentences_wu_sum
    all_sentences_wu_sum = []
    global all_sentences_wu_sum_divided
    all_sentences_wu_sum_divided = []
    for sentence_num in range(0,len(all_sentences001)): #現在第幾句
        wu_pulmer_sum = 0
        the_sentence = str(all_sentences001[sentence_num]).split()
        #print(the_sentence)
        timeframe_pairs_count = 0
        for words_num in range(0, (len(the_sentence)-1)): #標定第一個字
            if len(wordnet.synsets(the_sentence[words_num])) > 0 :
                syns1 = wordnet.synsets(the_sentence[words_num])
                syns1_num = len(syns1)
            else:
                continue
            for words_num2 in range((words_num+1), len(the_sentence)): #標定第二個字
                if len(wordnet.synsets(the_sentence[words_num2])) > 0 :
                    syns2 = wordnet.synsets(the_sentence[words_num2])
                    syns2_num = len(syns2)
                    for i in range(0, syns1_num):
                        for u in range(0, syns2_num):
                            #print(the_sentence[words_num],"的",syns1[i]," ", the_sentence[words_num2],"的",syns2[u]," " ,syns1[i].wup_similarity(syns2[u]))
                            wu_pulmer_sum += syns1[i].wup_similarity(syns2[u])
                            timeframe_pairs_count += 1
                else:
                    continue
        if timeframe_pairs_count != 0:
            all_sentences_wu_sum_divided.append(wu_pulmer_sum/timeframe_pairs_count) #算出除以pairs數量的平均值
        else: 
            all_sentences_wu_sum_divided.append(wu_pulmer_sum)
        all_sentences_wu_sum.append(wu_pulmer_sum)
        pair_counts.append(timeframe_pairs_count)
        #print(all_sentences_wu_sum)
        global wu_difference
        wu_difference = []
        for x in range(len(all_sentences_wu_sum)-1):
            wu_difference.append(all_sentences_wu_sum[x+1]-all_sentences_wu_sum[x])
        
    from sklearn import preprocessing

    results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = ['SBERT',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])
    
    if q<10:
        results_csv.to_csv(f"data/Text/00{q}({destination}).csv")
    else:
        results_csv.to_csv(f"data/Text/0{q}({destination}).csv")
    
    """ # for normalization
    wu_difference = np.array(wu_difference)
    SBERT = np.array(SBERT)
    #normalized_SBERT = preprocessing.normalize([SBERT])
    #normalized_wu_difference = preprocessing.normalize([wu_difference])


    min_max_scaler = preprocessing.MinMaxScaler()
    SBERT = np.reshape(SBERT,(479,1))
    wu_difference = np.reshape(wu_difference,(479,1))
    global normalized_SBERT
    global normalized_wu_difference
    normalized_SBERT  = min_max_scaler.fit_transform(SBERT)
    normalized_wu_difference  = min_max_scaler.fit_transform(wu_difference)
    """

參數表:<br>
all_sentences001: 所有的句子整合(沒有刪掉stop words)<br>
SBERT: 具有全部SBERT similarity的list<br>
all_sentences_wu_sum: 每句話的wu-pulmer similarity算出來的平均的list<br>
wu_difference: 兩句兩句減掉wu similarity的list<br>
normalized_SBERT: 就是表準化過後的SBERT list<br>
normalized_wu_difference: 標準化過後的wu_pulmer difference<br>
這邊注意wu-pulmer的different based應該不能這樣算

In [34]:
#calculate(f"data/Text/001.csv","results",10,1)

抓週期

In [35]:
def read_period(gcs): #所以gcs要是string
    df = pd.read_csv(gcs) 
    global period
    period = df["period(Wu)"][2] #現在是抓wu-pulmer divided 的週期
    #print(period)
    

In [36]:

x=19
q=x
freq = input("enter freq: ")
start = input("enter start time: ")
endt= input("enter end time:")
#read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")
cut_timeoffset = int(start)
cut_timeend = int(endt)
phase_num = 0
for t in range(cut_timeoffset,cut_timeoffset+int(period/2)):
    phase_num+=1
    print("start_time =", t)
    calculate(f"data/Text/0{x}(done).csv",f"halfpeakstep_phasef{freq}_{start}_{endt}_{t-cut_timeoffset}",period/2,period/2,t,end = cut_timeend) #,
print(phase_num)

start_time = 0
timeframe:  0 : 10 

0.48832798 

all right here's the story of my life and some future Ambitions in about eight minutes born 1999 to a my mom dad both from Ottawa so grew up in a bit of a little rural community outside of Ottawa dunrobin had 

have a brother and a sister came along my family's always been pretty close growing up makes pretty good memories as a family going up to the cottage going to visit my grandparents a lot had all four grandparents growing up so I got pretty close to them I would really describe 


timeframe:  1 : 11 

0.5541613 

have a brother and a sister came along my family's always been pretty close growing up makes pretty good memories as a family going up to the cottage going to visit my grandparents a lot had all four grandparents growing up so I got pretty close to them I would really describe 

my family is like a close-knit family we would go on lots of family vacations and stuff went to go see my grandparents down in Florida and what no

In [37]:
'''
start_subject = int(input("start_subject = "))
end_subject = int(input("end_subject = "))
for x in range(start_subject,end_subject+1):
    if x<10:
        try: 
            q=x
            read_period(f"data/peaks_of_PSD/00{x}(peaks).csv")
            for t in range(0,int(period/2)):
                print("start_time =", t)
                calculate(f"data/Text/00{x}(done).csv",f"halfpeakstep_phase0f{t}",period/2,period/2,t)
        except:
            print("error:",x)
            continue
        
    else:
        q=x
        try:
            read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")
            for t in range(0,int(period/2)):
                print("start_time =", t)
                calculate(f"data/Text/0{x}(done).csv",f"halfpeakstep_phase0f{t}",period/2,period/2,t)
        except:
            print("error:",x)
            continue
'''

'\nstart_subject = int(input("start_subject = "))\nend_subject = int(input("end_subject = "))\nfor x in range(start_subject,end_subject+1):\n    if x<10:\n        try: \n            q=x\n            read_period(f"data/peaks_of_PSD/00{x}(peaks).csv")\n            for t in range(0,int(period/2)):\n                print("start_time =", t)\n                calculate(f"data/Text/00{x}(done).csv",f"halfpeakstep_phase0f{t}",period/2,period/2,t)\n        except:\n            print("error:",x)\n            continue\n        \n    else:\n        q=x\n        try:\n            read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")\n            for t in range(0,int(period/2)):\n                print("start_time =", t)\n                calculate(f"data/Text/0{x}(done).csv",f"halfpeakstep_phase0f{t}",period/2,period/2,t)\n        except:\n            print("error:",x)\n            continue\n'

In [38]:
'''
start_subject = int(input("start_subject = "))
end_subject = int(input("end_subject = "))
for x in range(start_subject,end_subject+1):
    if x<10:
        try: 
            q=x
            read_period(f"data/peaks_of_PSD/00{x}(peaks).csv")
            calculate(f"data/Text/00{x}(done).csv","nonpeakstep",23,23)
        except:
            print("error:",x)
            continue
        
    else:
        q=x
        try:
            read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")
            calculate(f"data/Text/0{x}(done).csv","nonpeakstep",23,23)
        except:
            print("error:",x)
            continue
'''

'\nstart_subject = int(input("start_subject = "))\nend_subject = int(input("end_subject = "))\nfor x in range(start_subject,end_subject+1):\n    if x<10:\n        try: \n            q=x\n            read_period(f"data/peaks_of_PSD/00{x}(peaks).csv")\n            calculate(f"data/Text/00{x}(done).csv","nonpeakstep",23,23)\n        except:\n            print("error:",x)\n            continue\n        \n    else:\n        q=x\n        try:\n            read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")\n            calculate(f"data/Text/0{x}(done).csv","nonpeakstep",23,23)\n        except:\n            print("error:",x)\n            continue\n'

In [39]:
'''
for i in range(18,24):
    if i<10:
        try:
            calculate(f"data/Text/00{i}(done).csv")
            results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = ['SBERT',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])
            results_csv.to_csv(f'data/Text/00{i}(results).csv')
        except:
            print("error:", i)
            continue
    else:
        try:
            calculate(f"data/Text/0{i}(done).csv")
            results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = ['SBERT',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])
            results_csv.to_csv(f'data/Text/0{i}(results).csv')
        except:
            print("error:", i)
            continue
'''

'\nfor i in range(18,24):\n    if i<10:\n        try:\n            calculate(f"data/Text/00{i}(done).csv")\n            results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = [\'SBERT\',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])\n            results_csv.to_csv(f\'data/Text/00{i}(results).csv\')\n        except:\n            print("error:", i)\n            continue\n    else:\n        try:\n            calculate(f"data/Text/0{i}(done).csv")\n            results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = [\'SBERT\',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])\n            results_csv.to_csv(f\'data/Text/0{i}(results).csv\')\n        except:\n            print("error:", i)\n            continue\n'